In [ ]:
# Tested with scanpy 1.10.0, numpy 1.24.4, scipy 1.9.1, pandas 2.2.2, scikit-learn==1.5.1, and matplotlib 3.9.1
import pandas as pd
import numpy as np
import scanpy as sc
import scipy as sp
import matplotlib as mpl

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
sc.settings.verbosity = 0
sc.logging.print_header()
sc.set_figure_params(dpi=120, facecolor='white', figsize=(6, 6), vector_friendly=False)      # Set vector_friendly to True to rasterize the entire image 
mpl.rcParams['pdf.fonttype'] = 42                                                            # Vectorize words instead of letters

#### Load SG data to an anndata object 
- For the details about the design of AnnData object, please see [Getting started with anndata](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html) tutorial.

In [ ]:
project_folder = '.../Desktop/BasicResults/'
exp_file = 'CellxGene.csv'
coord_file = 'CellCoordinates.csv'
species = 'Mouse'                           # Optional
tissue = 'Embryonic Head'                            # Optional
preservation_method = 'Fixed frozen'        # Optional
panel_name = 'Craniofaicaldev'               # Optional

In [ ]:
# Load the cell-by-gene count matrix
adata = sc.read_csv(filename='{}/{}'.format(project_folder, exp_file), delimiter=',', first_column_names=True, dtype='int')
adata

In [ ]:
# Load the meta information
adata.uns['species'] = species
adata.uns['tissue'] = tissue
adata.uns['preservation_method'] = preservation_method
adata.uns['panel_name'] = panel_name
adata

In [ ]:
# Read the cell spatial metadata
cc = pd.read_csv('{}/{}'.format(project_folder, coord_file))
cc['label'] = cc['label'].astype('str')

In [ ]:
# Add cell spatial metadata to the obs table
adata.obs = adata.obs.merge(cc, left_index=True, right_on='label')
adata.obs

In [ ]:
# Add cell coordinates to the spatial slot of obsm table
coords = np.empty((len(cc.center_x),2), dtype=np.uint32)
for i in range(len(cc.center_x)):
    coords[i,:] = (cc.iloc[i].center_x, cc.iloc[i].center_y)
adata.obsm['spatial'] = coords

#### Check basic data stats

In [ ]:
# Sanity check the highest expressing genes
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True, log1p=True, percent_top=[50])

# Plot the number of genes per cell and total counts per cell
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=5, multi_panel=True)

In [ ]:
# Violin plot of area 
sc.pl.violin(adata, ['area'], jitter=5)

In [ ]:
# Check othe details about the raw data, as desired
# For example, check genes expressed in less than 1000 cells
adata.var.loc[adata.var['n_cells_by_counts'] < 1000]

In [ ]:
# Save the unpreprocessed anndata object to a h5ad file
filename = exp_file.split('.')[0]
adata.write_h5ad('{}/{}_unpreprocessed.h5ad'.format(project_folder, filename))

#### Preprocess the data to filter genes and cells

In [ ]:
# Identify appropriate cutoffs based on min/max transcripts and min/max area per cell
# Check the distribution plots above to identify cutoffs
# A method-based way to assign cutoffs is below (to use, uncomment the code first).
print('Min and max transcripts per cell: {}, {}'.format(np.min(adata.obs['total_counts']), np.max(adata.obs['total_counts'])))
print('Min and max area per cell: {}, {}'.format(np.min(adata.obs['area']), np.max(adata.obs['area'])))
min_tc_cutoff = 20
#max_tc_cutoff = 1200
min_area_cutoff = 347
#max_area_cutoff = 22000

In [ ]:
# Filter cells and genes based on cutoffs
sc.pp.filter_cells(adata, min_counts=min_tc_cutoff)     # Filter cells by min total transcripts per cell
#sc.pp.filter_cells(adata, max_counts=max_tc_cutoff)     # Filter cells by max total transcripts per cell
sc.pp.filter_cells(adata, min_genes=7)                 # Filter cells by min number of genes expressed
#sc.pp.filter_genes(adata, min_cells=1000)                # Filter genes by min number of cells expressed in
adata

In [ ]:
# Filter cells based on the area cutoffs
adata = adata[adata.obs['area'] > min_area_cutoff,:]    # Filter cells by min area per cell
#adata = adata[adata.obs['area'] < max_area_cutoff,:]    # Filter cells by max area per cell
adata

In [ ]:
# One way to determine the cutoffs is based on (median + ratio * MAD), a method used in the scRNAseq data analysis
# Please update the MAD ratios as needed
# median_tc = np.median(adata.obs['total_counts'])
# mad_tc = sp.stats.median_abs_deviation(adata.obs['total_counts'])
# median_area = np.median(adata.obs['area'])
# mad_area = sp.stats.median_abs_deviation(adata.obs['area'])
# print('median total counts: {}'.format(median_tc))
# print('MAD total counts: {}'.format(mad_tc))
# print('median area: {}'.format(median_area))
# print('MAD area: {}'.format(mad_area))

# min_tc_cutoff = median_tc - 2.5 * mad_tc
# max_tc_cutoff = median_tc + 8.0 * mad_tc
# min_area_cutoff = median_area - 3.0 * mad_area
# max_area_cutoff = median_area + 7.0 * mad_area
# print('min and max total count cutoff: {}, {}'.format(min_tc_cutoff, max_tc_cutoff))
# print('min and max area cutoff: {}, {}'.format(min_area_cutoff, max_area_cutoff))

In [ ]:
# Save the preprocessed anndata object to a h5ad file
adata.write_h5ad('{}/{}_preprocessed.h5ad'.format(project_folder, filename))

#### Normalize and log transform the data

In [ ]:
# Save raw counts as layer, then perform CPM normalization and log transformation
adata.layers["counts"] = adata.X.copy()     # preserve counts
# adata = adata.copy()                      # deepcopy the object, optional
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)
adata.raw = adata                           # freeze the state in .raw

In [ ]:
# Identifying highly-variable genes is optional depending on your panel size and is often not needed.
# sc.pp.highly_variable_genes(adata, n_top_genes=5)

In [ ]:
# Filter the adata with only highly variable genes
# adata = adata[:, adata.var.highly_variable]

#### Perform dimensionality reduction

In [ ]:
# Regression and scaling are optional as they are not recommended in the newest version of scanpy
# sc.pp.regress_out(adata, ['total_counts'])
sc.pp.scale(adata, max_value=10)            # optional
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata, color='Barx1')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

#### Compute and embed the neighborhood graph

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=30)

In [ ]:
sc.tl.umap(adata, min_dist=0.3)

#### Cluster the cells

In [ ]:
# Perform leiden clustering
sc.tl.leiden(adata, resolution=1)

In [ ]:
sc.pl.umap(adata, color=['leiden'], ncols=1, size=5)

#### Plot the cells spatially colored by the clusters

In [ ]:
sc.pl.spatial(adata, color=['leiden'], spot_size=100, palette=sc.pl.palettes.default_20)

#### Identify differentially expressed genes

In [ ]:
# Identify differential expressed genes
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.tl.dendrogram(adata, groupby="leiden")
sc.pl.rank_genes_groups_dotplot(adata, groupby="leiden", standard_scale="var", n_genes=4)

In [ ]:
# Display top 5 differential expressed genes for every cluster
pd.DataFrame(adata.uns["rank_genes_groups"]["names"]).head(5)

In [ ]:
# Save the anndata object to a h5ad file
adata.write_h5ad('{}/{}_leiden_reso_1.h5ad'.format(project_folder, filename))

In [ ]:
# Subset the AnnData object to include only specific clusters
adata_filter = adata[adata.obs['leiden'].isin(["1", "2", "3"...])] 

In [ ]:
adata_filter.write_h5ad('{}/{}_subset_leiden_reso_1.h5ad'.format(project_folder, filename))